## imports and files

In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import json
from copy import deepcopy
import random
import sys
import time

# get an absolute path to the directory that contains parent files
__file__ = globals()['_dh'][0]
project_dir = __file__ = globals()['_dh'][0]
sys.path.append(os.path.normpath(os.path.join(
    project_dir, '..', '..', '..')))

from experiments.utils.constants import (
    PIPLINES_PATH,
    NODE_PROFILING_CONFIGS_PATH,
    NODE_PROFILING_RESULTS_STATIC_PATH
)


In [4]:
series = 3
experiment_id = 3
config_key_mapper = "key_config_mapper.csv"
series_path = os.path.join(
    NODE_PROFILING_RESULTS_STATIC_PATH,
    'series',
    str(series))

## config map reader

In [5]:
def config_reader(csv_addr: str):
    config_path = os.path.join(
        series_path,
        csv_addr)
    key_config_df = pd.read_csv(config_path)
    return key_config_df
config_reader(config_key_mapper)

,experiment_id,pipeline_name,node_name,model_variant,cpu_request,memory_request,max_batch_size,max_batch_time,load,load_duration,series,series_meta,replicas
0,1,5-paper-video,resnet-human,resnet18,1,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
1,2,5-paper-video,resnet-human,resnet18,1,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
2,3,5-paper-video,resnet-human,resnet18,1,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
3,4,5-paper-video,resnet-human,resnet18,2,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
4,5,5-paper-video,resnet-human,resnet18,2,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,71,5-paper-video,resnet-human,resnet152,8,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
71,72,5-paper-video,resnet-human,resnet152,8,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
72,73,5-paper-video,resnet-human,resnet152,16,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
73,74,5-paper-video,resnet-human,resnet152,16,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1


In [6]:
def get_experiment_detail(experiment_id: int):
    path = os.path.join(
        series_path,        
        config_key_mapper
    )
    df = config_reader(config_key_mapper)
    return df[df["experiment_id"]==experiment_id]
get_experiment_detail(2)

,experiment_id,pipeline_name,node_name,model_variant,cpu_request,memory_request,max_batch_size,max_batch_time,load,load_duration,series,series_meta,replicas
1,2,5-paper-video,resnet-human,resnet18,1,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1


In [7]:
key_config_df = config_reader(config_key_mapper)
key_config_df

,experiment_id,pipeline_name,node_name,model_variant,cpu_request,memory_request,max_batch_size,max_batch_time,load,load_duration,series,series_meta,replicas
0,1,5-paper-video,resnet-human,resnet18,1,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
1,2,5-paper-video,resnet-human,resnet18,1,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
2,3,5-paper-video,resnet-human,resnet18,1,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
3,4,5-paper-video,resnet-human,resnet18,2,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
4,5,5-paper-video,resnet-human,resnet18,2,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,71,5-paper-video,resnet-human,resnet152,8,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
71,72,5-paper-video,resnet-human,resnet152,8,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
72,73,5-paper-video,resnet-human,resnet152,16,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
73,74,5-paper-video,resnet-human,resnet152,16,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1


In [8]:
def get_result_file_names(results_addr):
    files = []
    key_config = config_reader(config_key_mapper)
    experiments_keys = list(key_config["experiment_id"])
    for path in os.listdir(results_addr):
        if os.path.isfile(os.path.join(results_addr, path)):
            file_name = path.split(".")[0]
            try:
                if int(file_name) in experiments_keys:
                    files.append(path)
            except:
                pass
    return files
file_names = get_result_file_names(
    results_addr=series_path)
file_names

['38.json',
 '55.json',
 '69.json',
 '36.json',
 '6.json',
 '75.json',
 '52.json',
 '33.json',
 '11.json',
 '13.json',
 '9.json',
 '37.json',
 '45.json',
 '21.json',
 '50.json',
 '51.json',
 '54.json',
 '10.json',
 '25.json',
 '30.json',
 '29.json',
 '18.json',
 '62.json',
 '72.json',
 '61.json',
 '68.json',
 '56.json',
 '71.json',
 '66.json',
 '59.json',
 '70.json',
 '42.json',
 '4.json',
 '74.json',
 '63.json',
 '15.json',
 '20.json',
 '64.json',
 '73.json',
 '47.json',
 '67.json',
 '31.json',
 '8.json',
 '49.json',
 '1.json',
 '19.json',
 '3.json',
 '46.json',
 '28.json',
 '27.json',
 '22.json',
 '32.json',
 '23.json',
 '7.json',
 '17.json',
 '40.json',
 '5.json',
 '41.json',
 '60.json',
 '35.json',
 '2.json',
 '43.json',
 '14.json',
 '26.json',
 '24.json',
 '12.json',
 '57.json',
 '58.json',
 '44.json',
 '16.json',
 '39.json',
 '53.json',
 '48.json',
 '34.json',
 '65.json']

## Read resutls

In [9]:
def read_results(
    results_addr, return_type = "all", specific_file = None):
    files = get_result_file_names(results_addr=results_addr)
    
    if return_type == "all":
        return_results = {}
        for file in files:
            name = file.split(".")[0].split("/")[-1]
            full_path = os.path.join(
                results_addr, file
            )
            json_file = open(full_path)
            return_results[name] = json.load(json_file)
        return return_results

            
results = read_results(
    results_addr=series_path)
print(results.keys())


dict_keys(['38', '55', '69', '36', '6', '75', '52', '33', '11', '13', '9', '37', '45', '21', '50', '51', '54', '10', '25', '30', '29', '18', '62', '72', '61', '68', '56', '71', '66', '59', '70', '42', '4', '74', '63', '15', '20', '64', '73', '47', '67', '31', '8', '49', '1', '19', '3', '46', '28', '27', '22', '32', '23', '7', '17', '40', '5', '41', '60', '35', '2', '43', '14', '26', '24', '12', '57', '58', '44', '16', '39', '53', '48', '34', '65'])


In [10]:
def flatten_results(per_second_latencies):
    """
    change the results format from
    [[second_1], [second_2], ...]
    to:
    [req_1, req_2, ...]
    """
    flattend_results = []
    for second_results in per_second_latencies:
        for request_result in second_results:
            flattend_results.append(request_result)
    return flattend_results
flattened_results = flatten_results(results['1']['responses'])
len(flattened_results)

60

In [69]:
def latency_calculator(results):
    client_to_server_latencies = []
    model_latencies = []
    model_to_server_latencies = []
    latencies = {
        'client_to_server_latencies': [],
        'model_latencies': [],
        'model_to_server_latencies': [] 
    }
    timeout_count = 0
    for result in results:
        try:
            # outer times
            outter_times = result[
                'timing'] if 'timing' in result.keys() else result['time']
            sending_time = outter_times["sending_time"]
            arrival_time = outter_times["arrival_time"]
            # inner times
            data = result['outputs'][0]['data']
            data = json.loads(data[0])
            inner_times = data["time"]
            model_name = result['model_name']

            # TEMP to be fixed with a consistent time format
            inner_times = inner_times[model_name + '_times'][0]

            arrival_key = "arrival_" + model_name
            serve_key   = "serving_" + model_name
            model_arrival_time = inner_times[arrival_key]
            model_serving_time = inner_times[serve_key]
            # all three latencies
            client_to_server_latency =\
                model_arrival_time - sending_time
            model_latency =\
                model_serving_time - model_arrival_time
            model_to_server_latency =\
                arrival_time - model_serving_time
            client_to_server_latencies.append(client_to_server_latency)
            model_latencies.append(model_latency)
            model_to_server_latencies.append(model_to_server_latency)
            latencies = {
                'client_to_server_latencies': client_to_server_latencies,
                'model_latencies': model_latencies,
                'model_to_server_latencies': model_to_server_latencies
            }
        except KeyError:
            timeout_count += 1
    return latencies, timeout_count
experiment_id = 64
latencies, timeout_count = latency_calculator(
    flatten_results(results[str(experiment_id)]['responses']))
print(f'{timeout_count=}')
print(pd.DataFrame(latencies))

timeout_count=39
    client_to_server_latencies  model_latencies  model_to_server_latencies
0                     0.048265        22.886571                   0.315446
1                    21.901806        22.192502                   0.307734
2                    43.076366        15.699616                   0.204798
3                    58.203694        11.103679                   0.201899
4                    68.044231        17.897071                   0.201674
5                    85.309673         9.301476                   0.012162
6                    93.629958        10.101213                   0.104969
7                   103.639539         8.104300                   0.103414
8                   110.714560         9.205893                   0.200522
9                   119.976758        19.499093                   0.306861
10                  138.743904        15.699247                   0.402652
11                  153.242199        17.599520                   0.205070
12      

In [66]:
def metric_summary(metric, values):
    summary = {}
    if values != []:
        summary[f'{metric}_avg'] = np.average(values)
        summary[f'{metric}_p99'] = np.percentile(values, 99)
        summary[f'{metric}_p50'] = np.percentile(values, 50)
        summary[f'{metric}_var'] = np.var(values)
        summary[f'{metric}_max'] = max(values)
        summary[f'{metric}_min'] = min(values)
    else:
        summary[f'{metric}_avg'] = None
        summary[f'{metric}_p99'] = None
        summary[f'{metric}_p50'] = None
        summary[f'{metric}_var'] = None
        summary[f'{metric}_max'] = None
        summary[f'{metric}_min'] = None
    return summary

def latency_summary(latencies):
    summary = {}
    for metric_name, values in latencies.items():
        summary.update(
            metric_summary(
                metric=metric_name, values=values))
    return summary

summary = latency_summary(latencies=latencies)
pd.DataFrame(list(summary.items()))

def result_processing(experiment_id):
    results = read_results(results_addr=series_path)
    final_dataframe = []
    result = results[str(experiment_id)]
    processed_exp = {'experiment_id': experiment_id}
    latencies = latency_calculator(
        flatten_results(results[str(experiment_id)]['responses']))
    latencies = latency_summary(latencies)
    processed_exp.update(latencies)
    processed_exp['start_time'] = time.ctime(
        result['start_time_experiment'])
    processed_exp['end_time'] = time.ctime(
        result['end_time_experiment'])
    processed_exp['duration'] = round(
        result['end_time_experiment'] - result['start_time_experiment'])
    skipped_metrics = [
        'time_cpu_usage_count',
        'time_cpu_usage_rate',
        'time_cpu_throttled_count',
        'time_cpu_throttled_rate',
        'time_memory_usage',
        'time_throughput',
        'responses',
        'start_time_experiment',
        'end_time_experiment'   
    ]
    for metric, values in result.items():
        if metric in skipped_metrics:
            continue
        processed_exp.update(metric_summary(metric=metric, values=values))
    final_dataframe.append(processed_exp)
    return pd.DataFrame(final_dataframe)
experiment_id = 64
output = result_processing(experiment_id)

KeyError: 'time'

In [16]:
output.columns

Index(['experiment_id', 'client_to_server_latencies_avg',
       'client_to_server_latencies_p99', 'client_to_server_latencies_p50',
       'client_to_server_latencies_var', 'client_to_server_latencies_max',
       'client_to_server_latencies_min', 'model_latencies_avg',
       'model_latencies_p99', 'model_latencies_p50', 'model_latencies_var',
       'model_latencies_max', 'model_latencies_min',
       'model_to_server_latencies_avg', 'model_to_server_latencies_p99',
       'model_to_server_latencies_p50', 'model_to_server_latencies_var',
       'model_to_server_latencies_max', 'model_to_server_latencies_min',
       'start_time', 'end_time', 'duration', 'cpu_usage_count_avg',
       'cpu_usage_count_p99', 'cpu_usage_count_p50', 'cpu_usage_count_var',
       'cpu_usage_count_max', 'cpu_usage_count_min', 'cpu_usage_rate_avg',
       'cpu_usage_rate_p99', 'cpu_usage_rate_p50', 'cpu_usage_rate_var',
       'cpu_usage_rate_max', 'cpu_usage_rate_min', 'cpu_throttled_count_avg',
       'cp

In [17]:
experiment_id = 14
latencies = latency_calculator(
    flatten_results(results[str(experiment_id)]['responses']))
print(pd.DataFrame(latencies))

    client_to_server_latencies  model_latencies  model_to_server_latencies
0                     0.035364         0.184394                   0.009770
1                     0.027726         0.066673                   0.009074
2                     0.020905         0.095008                   0.010575
3                     0.034425         0.042324                   0.008526
4                     0.019615         0.181757                   0.007569
5                     0.020600         0.063255                   0.008383
6                     0.021779         0.155235                   0.009538
7                     0.019918         0.066117                   0.008780
8                     0.022339         0.066038                   0.009091
9                     0.023826         0.173868                   0.008833
10                    0.018403         0.061095                   0.006266
11                    0.023032         0.054457                   0.009538
12                    0.0

In [51]:
key_config_df = config_reader(config_key_mapper)
key_config_df =  key_config_df[key_config_df['model_variant'] == 'resnet152']
key_config_df

,experiment_id,pipeline_name,node_name,model_variant,cpu_request,memory_request,max_batch_size,max_batch_time,load,load_duration,series,series_meta,replicas
60,61,5-paper-video,resnet-human,resnet152,1,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
61,62,5-paper-video,resnet-human,resnet152,1,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
62,63,5-paper-video,resnet-human,resnet152,1,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
63,64,5-paper-video,resnet-human,resnet152,2,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
64,65,5-paper-video,resnet-human,resnet152,2,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
65,66,5-paper-video,resnet-human,resnet152,2,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
66,67,5-paper-video,resnet-human,resnet152,4,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
67,68,5-paper-video,resnet-human,resnet152,4,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
68,69,5-paper-video,resnet-human,resnet152,4,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1
69,70,5-paper-video,resnet-human,resnet152,8,60Gi,1,1,1,60,3,specification: initial test to check the frame...,1


In [64]:
experiment_id = 61
output = result_processing(experiment_id)
selected_metrics = [
    'model_latencies_p99',
    'cpu_usage_rate_avg',
    'throughput_avg'
]
output[selected_metrics]

KeyError: 'time'

In [53]:
experiment_id = 64
output = result_processing(experiment_id)
selected_metrics = [
    'model_latencies_p99',
    'cpu_usage_rate_avg',
    'throughput_avg'
]
output[selected_metrics]

KeyError: 'time'

In [61]:
experiment_id = 67
output = result_processing(experiment_id)
selected_metrics = [
    'model_latencies_p99',
    'cpu_usage_rate_avg',
    'throughput_avg'
]
output[selected_metrics]

,model_latencies_p99,cpu_usage_rate_avg,throughput_avg
0,4.002633,1.570571,0.229291


In [59]:
experiment_id = 70
output = result_processing(experiment_id)
selected_metrics = [
    'model_latencies_p99',
    'cpu_usage_rate_avg',
    'throughput_avg'
]
output[selected_metrics]

,model_latencies_p99,cpu_usage_rate_avg,throughput_avg
0,1.465754,0.447612,0.25533


In [60]:
experiment_id = 73
output = result_processing(experiment_id)
selected_metrics = [
    'model_latencies_p99',
    'cpu_usage_rate_avg',
    'throughput_avg'
]
output[selected_metrics]

,model_latencies_p99,cpu_usage_rate_avg,throughput_avg
0,0.539474,0.256253,0.253506
